In [1]:
import pandas as pd
import numpy as np
import os
import time
import pickle
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Login

In [3]:
synergy_url = 'https://www.synergysportstech.com'
app_url = 'https://apps.synergysports.com/basketball'

In [4]:
def login(driver, cred, synergy_url=synergy_url):
    driver.get(f'{synergy_url}/synergy/')
    time.sleep(5)
    # Get username & password
    with open(cred, 'r') as f:
        user_password = f.readlines()[0].replace('\n', '').split(', ')
    # Login
    userfield = driver.find_element_by_id('Username')
    userfield.send_keys(user_password[0])
    passwordfield = driver.find_element_by_id('Password')
    passwordfield.send_keys(user_password[1])
    driver.find_element_by_xpath("//button[@class='btn btn-primary']").click()

In [5]:
# driver = webdriver.Chrome('/usr/local/bin/chromedriver')
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)



====== WebDriver manager ======
Current google-chrome version is 114.0.5735
Get LATEST chromedriver version for 114.0.5735 google-chrome
There is no [mac64] chromedriver for browser 114.0.5735 in cache
Trying to download new driver from https://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_mac64.zip
Driver has been saved in cache [/Users/xulian/.wdm/drivers/chromedriver/mac64/114.0.5735.90]


In [6]:
cred = '../synergy_cred.txt'
login(driver, cred)

## Scrape Setup

In [7]:
downloads_dir = '../../../../Downloads'

### Situations

In [9]:
situations_dict = {
    'Pick and Rolls Including Passes': '101',
    'Isolations Including Passes': '100',
    'Post-Ups Including Passes': '102',
    'Catch and Shoot': '116',
    'Isolation': '10',
    'P&R Ball Handler': '11',
    'Post-Up': '15',
    'P&R Roll Man': '31',
    'Spot Up': '50',
    'Off Screen': '67',
    'Hand Off': '78',
    'Cut': '81',
    'Offensive Rebounds': '89',
    'Transition': '92',
    'Miscellaneous': '124',
    'All field goal attempts': '99',
    'At Rim': '127',
    'Long (3 point jump shots)': '109',
    'All Jump Shots off the Dribble': '110'
}

### Functions

In [10]:
def create_dirs(league_dict):
    league = league_dict['league']
    for season in league_dict['seasonId'].keys():
        dirname = f'../data/{league}_{season}'
        if dirname not in os.listdir():
            os.mkdir(dirname)

In [11]:
def get_stats_sheet(situation, 
                    reportType, division, season,
                    league_dict, 
                    output_format='xlsx',
                    situations_dict=situations_dict,
                    driver=driver):
    sheet_url = (
        f'{app_url}/'
        'leaderboards?'
        f'leagueId={league_dict["leagueId"]}&'
        f'seasonId={league_dict["seasonId"][season]}&'
        f'conferenceIds={league_dict["conferenceIds"][division]}&'
        f'reportType={league_dict["reportType"][reportType]}&'
        f'situation={situations_dict[situation]}&'
        'perGame=0&cutoff=0'
    )
    driver.get(sheet_url)
    time.sleep(20)
    
    # Download
    xpath = "//div[@class='ng-select']"
    if output_format == 'csv':
        xpath = "//div[@class='ng-select ml-2']"
        
    for b in driver.find_elements_by_xpath(xpath):
        if b.text == output_format.upper():
            b.click()
    
    time.sleep(10)

In [12]:
def download_season(season, 
                    reportType, division, 
                    league_dict, 
                    output_format='xlsx',
                    situations_dict=situations_dict,
                    driver=driver):
    print(f'Downloading Season {season}')
    for situation in tqdm(situations_dict.keys()):
        get_stats_sheet(situation=situation,
                        reportType=reportType,
                        division=division,
                        season=season,
                        league_dict=league_dict,
                        output_format=output_format)
    print('Done.')

In [13]:
def move_season(season, league_dict, 
                downloads_dir=downloads_dir):
    league = league_dict["league"]
    filename_prefix = (
        'Leaderboards - '
        f'{league} '
        f'{season} '
    )
    
    new_dir = f'../data/{league}_{season}'
    for fn in os.listdir(downloads_dir):
        if fn.startswith(filename_prefix):
            os.rename(f'{downloads_dir}/{fn}',
                      f'{new_dir}/{fn}')
            
    print(f'{new_dir}: {len(os.listdir(new_dir))}')

In [14]:
def find_missing_sheets(season, division, reportType, league,
                        output_format, league_dict, 
                        situations_dict=situations_dict,
                        driver=driver,
                        downloads_dir=downloads_dir):
    for situation in situations_dict.keys():
        miss_flag = 1
        for f in os.listdir(f'../data/{league}_{season}'):
            if situation in f:
                miss_flag = 0
                break
        if miss_flag == 1:
            print(situation)
            get_stats_sheet(situation=situation,
                            reportType=reportType,
                            division=division,
                            season=season,
                            league_dict=league_dict, 
                            output_format=output_format)
    move_season(season=season, 
                league_dict=league_dict, 
                division=division)

## Scrape College Men

In [15]:
ncaa_dict = {
    'league': 'College Men',
    'leagueId': '54457dce300969b132fcfb37',
    'seasonId': {
        '2022-2023': '630530b90a41b857ff3c1500',
        '2021-2022': '6085b5d0e6c2413bc4ba9122',
        '2020-2021': '5dad07257455336e8774eb66',
        '2019-2020': '5d56fb68f52909811e1c65e5',
        '2018-2019': '5b4bd3ab9559b26e7e107793',
        '2017-2018': '59af08c917120e9c9a97983b'
    },
    'conferenceIds': {
        'NCAA Division I': '54457dce300969b132fcfb4a'
    }, 
    'reportType': {
        'Player Offensive': '0',
        'Player Defensive': '1',
        'Team Offensive': '2',
        'Team Defensive': '3'
    }
}

In [17]:
create_dirs(ncaa_dict)

In [18]:
division = 'NCAA Division I'
reportType = 'Player Offensive'
output_format = 'csv'

### 2022-2023

In [19]:
league = 'College Men'

In [20]:
season = '2022-2023'

In [21]:
download_season(season=season,
                reportType=reportType,
                division=division,
                league_dict=ncaa_dict,
                output_format=output_format)
move_season(season=season, 
            league_dict=ncaa_dict)

100%|███████████████████████████████████████████| 19/19 [09:46<00:00, 30.85s/it]

Done.
../data/College Men_2022-2023: 19


### 2021-2022

In [19]:
season = '2021-2022'

In [22]:
download_season(season=season,
                reportType=reportType,
                division=division,
                league_dict=ncaa_dict,
                output_format=output_format)
move_season(season=season, 
            league_dict=ncaa_dict)

### 2020-2021

In [32]:
season = '2020-2021'

In [33]:
download_season(season=season,
                reportType=reportType,
                division=division,
                league_dict=ncaa_dict,
                output_format=output_format)
move_season(season=season, 
            league_dict=ncaa_dict)

100%|███████████████████████████████████████████| 19/19 [09:55<00:00, 31.34s/it]

Done.
../data/College Men_2020-2021: 19


### 2019-2020

In [34]:
season = '2019-2020'

In [35]:
download_season(season=season,
                reportType=reportType,
                division=division,
                league_dict=ncaa_dict,
                output_format=output_format)
move_season(season=season, 
            league_dict=ncaa_dict)

100%|███████████████████████████████████████████| 19/19 [09:54<00:00, 31.28s/it]

Done.
../data/College Men_2019-2020: 19


### 2018-2019

In [36]:
season = '2018-2019'

In [37]:
download_season(season=season,
                reportType=reportType,
                division=division,
                league_dict=ncaa_dict,
                output_format=output_format)
move_season(season=season, 
            league_dict=ncaa_dict)

100%|███████████████████████████████████████████| 19/19 [09:53<00:00, 31.24s/it]

Done.
../data/College Men_2018-2019: 19


### 2017-2018

In [38]:
season = '2017-2018'

In [39]:
download_season(season=season,
                reportType=reportType,
                division=division,
                league_dict=ncaa_dict,
                output_format=output_format)
move_season(season=season, 
            league_dict=ncaa_dict)

100%|███████████████████████████████████████████| 19/19 [09:54<00:00, 31.27s/it]

Done.
../data/College Men_2017-2018: 19


In [22]:
driver.close()